# 2. Projects operations system


Este proyecto realiza operaciones sobre los mails y los etiqueta segun el proyecto y el objeto del mail. 
En este caso, lee un email, identifica de que proyecto se trata, identifica cual es el objeto del email y realiza la acción indicada en cada caso

1. Utiliza Langchain agents.
1. Utiliza MCP
1. Utiliza langflow para el logging
1. No requiere suscriciones, Utiliza solamente modelos de LLM locales y ligeros, en este caso usamos qwen3:7b sobre ollama.


In [ ]:
# %%capture --no-stderr
%pip install -U --upgrade pip langchain dotenv langchain_ollama langfuse pandas langgraph mcp langchain_mcp_adapters

In [1]:
from langfuse.langchain import CallbackHandler
from dotenv import load_dotenv
import os

# load environment variables from .env file
load_dotenv()
workfolder = os.getenv('WORKFOLDER')

# Initialize Langfuse CallbackHandler for LangGraph/Langchain (tracing)
langfuse_handler = CallbackHandler() 
llm_config = {"configurable": {"thread_id": "abc123"}, "recursion_limit": 10, "callbacks": [langfuse_handler]}

In [11]:
import pandas as pd

projects = [
    {
        "tag": "APCSC",
        "name": "Chatbot de Atención al Cliente Potenciado por IA",
        "description": "Creación de un chatbot de atención al cliente potenciado por IA que pueda manejar una amplia gama de consultas de clientes y proporcionar respuestas instantáneas. \
            El chatbot se integrará con el sistema de atención al cliente existente de la empresa y utilizará el procesamiento del lenguaje natural (PNL) para comprender y responder a las consultas de los clientes con precisión. \
            El chatbot será entrenado con datos históricos de atención al cliente para garantizar que pueda manejar los problemas comunes de manera efectiva.",
        "status": "Actualmente entrenando al chatbot con las últimas transcripciones de servicio al cliente.",
    },
    {
        "tag": "REMP",
        "name": "Plataforma de Monitoreo de Energía Renovable",
        "description": "Desarrollo de una plataforma de monitoreo de energía renovable que permite a los usuarios rastrear su consumo y producción de energía en tiempo real. \
            La plataforma incluirá características como análisis de uso de energía, seguimiento del rendimiento de los paneles solares y monitoreo del almacenamiento de baterías. \
            La plataforma estará diseñada para ser fácil de usar y proporcionará información práctica para ayudar a los usuarios a reducir su consumo de energía y disminuir su huella de carbono.",
        "status": "Realización de pruebas de usuario en la interfaz de la plataforma y la visualización de datos.",
    },
    {
        "tag": "OLPC",
        "name": "Plataforma de Aprendizaje en Línea para Programación",
        "description": "Desarrollo de una plataforma de aprendizaje en línea para programación que proporciona cursos interactivos, desafíos de codificación y retroalimentación en tiempo real a los estudiantes. \
            La plataforma estará diseñada para ser accesible a usuarios de todos los niveles de habilidad, desde principiantes hasta programadores avanzados. \
            Incluirá características como tutoriales en video, ejercicios de codificación y un foro comunitario para que los estudiantes se conecten y colaboren.",
        "status": "Implementando nuevos desafíos de codificación para mejorar la participación del usuario.",
    }
]

email_reasons = [
    {
        "reason": "actualización",
        "description": "Alguna tarea del proyecto ha finalizado",
        "action": "actualizar el estado del proyecto y notificar por correo electrónico al pm con el nuevo estado",
    }, 
    {
        "reason": "problema",
        "description": "Ha ocurrido algo que puede retrasar el proyecto",
        "action": "preguntar por correo electrónico al analista y al técnico que investiguen el problema y poner al pm en copia"
    },
    {
        "reason": "solicitud",
        "description": "Se ha solicitado algo que cambia el proyecto",
        "action": "preguntar por correo electrónico al analista que evalúe los cambios y pedir la aprobación al pm"
    },
]

projects_df = pd.DataFrame(projects)
projects_df.to_csv(os.path.join(workfolder, "projects.csv"), index=False)
projects_df = pd.read_csv(os.path.join(workfolder, "projects.csv"))

projects_df.head()


,tag,name,description,status
0,APCSC,Chatbot de Atención al Cliente Potenciado por IA,Creación de un chatbot de atención al cliente ...,Actualmente entrenando al chatbot con las últi...
1,REMP,Plataforma de Monitoreo de Energía Renovable,Desarrollo de una plataforma de monitoreo de e...,Realización de pruebas de usuario en la interf...
2,OLPC,Plataforma de Aprendizaje en Línea para Progra...,Desarrollo de una plataforma de aprendizaje en...,Implementando nuevos desafíos de codificación ...


In [ ]:
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent

simple_agent = create_react_agent(name="simple_agent", model=ChatOllama(model="qwen3"), 
    tools=[], response_format='json', prompt="You are a helpful assistant.")

def gen_email(reason_desc, project_desc, from_name, from_email, to_email, lenguaje ):

    input_message = {"role": "user", 
        "content": f"""
        Explain a creative 100 word issue related to a project. 
        - issue reason: {reason_desc} 
        - project description: {project_desc}, 
        
        Do not explain the project description, just explain what part of the project is affected
        Use profesional greeting, signature, farewell in the email, write in multiple lines.

        email information:
        - from_name: {from_name}
        - from_email: {from_email}
        - to: {to_email}
        - email language: {lenguaje}
        - output format json: {{ "from_email": "", "to": "", "subject": "", "body": "" }}
        /no_think"""
    }

    last_message = ""
    # Use the agent
    for step in simple_agent.stream(
        {"messages": [input_message]}, llm_config, stream_mode="values"
    ):
        last_message = step["messages"][-1]
        last_message.pretty_print()
    return last_message.content

gen_email("Something has happened that can put the project on hold", "E-comerce with a package location module", "John Doe","john.doe@mycompany.com", 'myself@mycompany.com', 'spanish')

In [ ]:
dummy_emails = []

for idx, project in projects_df.iterrows():
  project = project.to_dict()
  for reason in email_reasons:
    result = gen_email(reason["description"], project['description'], 'John Doe', 'john.doe@mycompany.com', 'myself@mycompany.com','spanish' )
    
    generated_mail = {
      'content': result,
      'real_project': project['tag'],
      'real_reason': reason["reason"]
    }

    dummy_emails.append(generated_mail)

dummy_emails

In [ ]:
import re
import json

think_rx = re.compile(r"<think>.*?</think>\s*", re.IGNORECASE | re.DOTALL)

dummy_emails = [
 {'content': '<think>\n\n</think>\n\n{\n  "from_email": "john.doe@mycompany.com",\n  "to": "myself@mycompany.com",\n  "subject": "Actualización del Proyecto: Chatbot de Atención al Cliente",\n  "body": "Buen día,\\n\\nQuiero informarle que una parte del proyecto de desarrollo del chatbot de atención al cliente ha finalizado exitosamente. Se completó el entrenamiento del modelo de inteligencia artificial con los datos históricos de consultas, lo que permitirá al chatbot comprender y responder a las preguntas de los clientes de manera precisa y eficiente.\\n\\nEste logro representa un avance significativo en la integración del chatbot con el sistema existente de atención al cliente.\\n\\nAtentamente,\\nJohn Doe\\n\\nSaludos cordiales."\n}',
  'real_project': 'APCSC',
  'real_reason': 'actualización'},
 {'content': '<think>\n\n</think>\n\n{\n  "from_email": "john.doe@mycompany.com",\n  "to": "myself@mycompany.com",\n  "subject": "Retraso en el desarrollo del chatbot de atención al cliente",\n  "body": "Buen día,\\n\\nLamento informarle que ha surgido un retraso en la integración del chatbot con el sistema de atención al cliente existente. Este retraso se debe a dificultades técnicas en la sincronización de los datos históricos de consultas, lo que afecta la capacidad del chatbot para comprender y responder de manera precisa a las consultas de los clientes.\\n\\nEste problema podría impactar el cronograma previsto para la entrega funcional del proyecto. Estamos trabajando activamente para resolverlo lo antes posible.\\n\\nAgradezco su atención y quedo atento a sus comentarios.\\n\\nSaludos cordiales,\\nJohn Doe"\n}',
  'real_project': 'APCSC',
  'real_reason': 'problema'},
 {'content': '<think>\n\n</think>\n\n{\n  "from_email": "john.doe@mycompany.com",\n  "to": "myself@mycompany.com",\n  "subject": "Nueva solicitud que afecta al proyecto de chatbot",\n  "body": "Buen día,\\n\\nSe ha solicitado una modificación al proyecto de desarrollo del chatbot de atención al cliente, que implica un cambio en la integración con el sistema existente. Esta solicitud afecta directamente la arquitectura del chatbot y su capacidad para manejar consultas con precisión mediante el procesamiento del lenguaje natural. Se requiere revisar cómo esta modificación impactará en el entrenamiento del modelo y en la entrega del proyecto.\\n\\nAtentamente,\\nJohn Doe\\n\\nSaludos cordiales."\n}',
  'real_project': 'APCSC',
  'real_reason': 'solicitud'},
 {'content': '<think>\n\n</think>\n\n{\n  "from_email": "john.doe@mycompany.com",\n  "to": "myself@mycompany.com",\n  "subject": "Actualización del Proyecto de Plataforma de Monitoreo de Energía Renovable",\n  "body": "Buen día,\\n\\nQuiero informarle que una tarea clave del proyecto ha finalizado exitosamente. Se completó el desarrollo del módulo de análisis de uso de energía, el cual permite a los usuarios obtener una visión detallada de su consumo y producción en tiempo real. Este avance facilita la integración de los demás componentes del sistema, como el seguimiento del rendimiento de los paneles solares y el monitoreo del almacenamiento de baterías.\\n\\nAgradecemos su atención.\\nAtentamente,\\nJohn Doe\\n\\nHasta pronto."\n}',
  'real_project': 'REMP',
  'real_reason': 'actualización'},
 {'content': '<think>\n\n</think>\n\n{\n  "from_email": "john.doe@mycompany.com",\n  "to": "myself@mycompany.com",\n  "subject": "Retraso en el desarrollo de la plataforma de monitoreo de energía renovable",\n  "body": "Buen día,\\n\\nHa ocurrido un retraso en el desarrollo de la plataforma de monitoreo de energía renovable, específicamente en la integración del módulo de análisis de uso de energía. Este componente presenta dificultades técnicas en la sincronización de datos en tiempo real con los sensores de los paneles solares, lo que podría afectar la funcionalidad central de la plataforma.\\n\\nQuedo atento a su respuesta.\\n\\nSaludos cordiales,\\nJohn Doe"\n}',
  'real_project': 'REMP',
  'real_reason': 'problema'},
 {'content': '<think>\n\n</think>\n\n{\n  "from_email": "john.doe@mycompany.com",\n  "to": "myself@mycompany.com",\n  "subject": "Nueva solicitud que afecta al proyecto",\n  "body": "Buen día,\\n\\nSe ha solicitado una nueva funcionalidad que cambia el proyecto original. La plataforma de monitoreo de energía renovable ahora incluirá un módulo de predicción de demanda energética basado en datos históricos y condiciones climáticas. Esta solicitud afecta directamente al desarrollo del análisis de uso de energía y al diseño de la interfaz de usuario, ya que requiere integración de algoritmos de inteligencia artificial y nuevos datos de entrada.\\n\\nAtentamente,\\nJohn Doe\\n\\nCordialmente,"\n}',
  'real_project': 'REMP',
  'real_reason': 'solicitud'},
 {'content': '<think>\n\n</think>\n\n{\n  "from_email": "john.doe@mycompany.com",\n  "to": "myself@mycompany.com",\n  "subject": "Actualización del Proyecto de Plataforma de Aprendizaje",\n  "body": "Buen día,\\n\\nQuiero informarle que una tarea clave del proyecto ha finalizado exitosamente. Se completó el desarrollo del módulo de retroalimentación en tiempo real para los ejercicios de codificación. Este componente es fundamental para garantizar que los estudiantes reciban respuestas inmediatas y precisas al resolver los desafíos, lo que mejora significativamente su experiencia de aprendizaje.\\n\\nAgradecemos su atención y quedamos atentos para continuar con las siguientes etapas.\\n\\nSaludos cordiales,\\nJohn Doe\\n\\nAtentamente,\\nJohn Doe\\n\\nHasta pronto."\n}',
  'real_project': 'OLPC',
  'real_reason': 'actualización'},
 {'content': '<think>\n\n</think>\n\n{\n  "from_email": "john.doe@mycompany.com",\n  "to": "myself@mycompany.com",\n  "subject": "Retraso en el desarrollo de la plataforma de aprendizaje en línea",\n  "body": "Buenos días,\\n\\nLamento informarle que ha surgido un retraso en el desarrollo de la plataforma de aprendizaje en línea. El problema se debe a la necesidad de reestructurar el sistema de retroalimentación en tiempo real, que está más complejo de lo previsto. Esta parte es crucial para garantizar una experiencia de usuario fluida y efectiva.\\n\\nEstamos trabajando para minimizar el impacto y mantener el cronograma lo más cercano posible.\\n\\nAtentamente,\\nJohn Doe\\n\\nHasta pronto."\n}',
  'real_project': 'OLPC',
  'real_reason': 'problema'},
 {'content': '<think>\n\n</think>\n\n{\n  "from_email": "john.doe@mycompany.com",\n  "to": "myself@mycompany.com",\n  "subject": "Nueva solicitud que afecta el proyecto de la plataforma de aprendizaje en línea",\n  "body": "Hola, \\n\\nSe ha solicitado la incorporación de una función de inteligencia artificial para personalizar el contenido de aprendizaje de cada estudiante, lo cual cambia el proyecto original. Esta característica requiere ajustes en el diseño del sistema, la integración de nuevos algoritmos y la revisión de la arquitectura tecnológica. Además, afecta la planificación de recursos y el cronograma del proyecto. \\n\\nAtentamente, \\nJohn Doe \\n\\nCordialmente,"\n}',
  'real_project': 'OLPC',
  'real_reason': 'solicitud'}]

for dummy in dummy_emails:
    dummy['content'] = think_rx.sub("", dummy['content'])
    dummy['content'] = dict(json.loads(dummy['content']))
    dummy['from_email'] = dummy['content']['from_email']
    dummy['to'] = dummy['content']['to']
    dummy['subject'] = dummy['content']['subject']
    dummy['body'] = dummy['content']['body']
    del dummy['content']
    
dummy_emails_df = pd.DataFrame(dummy_emails)
dummy_emails_df.to_csv(os.path.join(workfolder, "dummy_emails.csv"), index=False)

dummy_emails_df

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
client = MultiServerMCPClient(
    {
        "pm_operations": {
            "command": "python",
            # Make sure to update to the full absolute path to your math_server.py file
            "args": ["/home/daimler/workspaces/agents-course-huggingface/langagent/tools/pm_operations.py"],
            "transport": "stdio",
        },
    }
)
pm_operations_tools = await client.get_tools()

import sys
sys.path.append("/home/daimler/workspaces/agents-course-huggingface/langagent/tools")
from pm_operations import get_emails 
from pm_operations import get_projects 
from pm_operations import classify_tag_email 
pm_operations_tools = [get_emails, get_projects]

pm_operations_tools

[StructuredTool(name='get_emails', description="Returns the last email received by the system in json: {'from_email':'','to':'','subject':'','body':''}.", args_schema=<class 'langchain_core.utils.pydantic.get_emails'>, func=<function get_emails at 0x76ae3a6cfe20>),
 StructuredTool(name='get_projects', description="Returns the list of the projects in json: [{'tag':'','name':'','description':''}]", args_schema=<class 'langchain_core.utils.pydantic.get_projects'>, func=<function get_projects at 0x76ae3a6ceb60>)]

In [9]:
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent

simple_agent = create_react_agent(name="simple_agent", model=ChatOllama(model="qwen3:14b"), 
    tools=pm_operations_tools, response_format='json', prompt="You are a helpful assistant.")

input_message = {"role": "user", "content": "Read the last email and decide wich project is it about. outpuf format: {'project_tag':'', 'project_name':'', email_subject':'' }" }

# Use the agent
for step in simple_agent.stream({"messages": [input_message]}, llm_config, stream_mode="values"):
    step["messages"][-1].pretty_print()


================================ Human Message =================================

Read the last email and decide wich project is it about. outpuf format: {'project_tag':'', 'project_name':'', email_subject':'' }
================================== Ai Message ==================================
Name: simple_agent

<think>
Okay, let's tackle this user query. The user wants me to read the last email and determine which project it's about, then output a specific JSON format.

First, I need to get the last email. The available functions include get_emails, which returns the last email in JSON. So I should call get_emails first. Once I have the email details, I need to check the subject and body to figure out which project it's related to. 

But wait, how do I know which project? The get_projects function can provide a list of projects with their tags, names, and descriptions. So after retrieving the email, I should call get_projects to get the list of projects. Then, I can match the email's c

In [ ]:
from langchain_core.tools import tool
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent

@tool
def get_reasons() -> dict:
    """ Returns the list of reasons for sending an email in json: [{ 'reason':'','description':'','action':'' }]. 
    """
    return email_reasons

simple_agent = create_react_agent(name="simple_agent", model=ChatOllama(model="qwen3:14b"), 
    tools=[get_emails, get_reasons], response_format='json', prompt="You are a helpful assistant.")

input_message = {"role": "user", "content": "Read the last email and decide wich reason is it for. outpuf format: {'reason':'', email_subject':'' }" }

# Use the agent
for step in simple_agent.stream({"messages": [input_message]}, llm_config, stream_mode="values"):
    step["messages"][-1].pretty_print()


================================ Human Message =================================

Read the last email and decide wich reason is it for. outpuf format: {'reason':'', email_subject':'' }
================================== Ai Message ==================================
Name: simple_agent

<think>
Okay, the user wants me to read the last email and determine which reason it corresponds to. The output should be a JSON object with 'reason' and 'email_subject'.

First, I need to get the last email. The available functions are get_emails and get_reasons. The get_emails function returns the last email's details, including the subject. So I should call get_emails first to retrieve the email's subject and other details.

Once I have the email's subject, I need to figure out which reason it belongs to. The get_reasons function provides a list of reasons, each with a description and action. However, the user hasn't provided specific reasons, so maybe the reasons are predefined. Wait, the function get

In [ ]:

import pandas as pd

stakeholders = [
    {"name": "Diana Miller", "phone": "555-456-7890", "email": "diana.miller@cityhall.com", "company": "cityhall", "vip": False, "prefered_languaje": "Italian", "role": "technician"},
    {"name": "Henry Green", "phone": "555-890-1234", "email": "henry.green@cityhall.com", "company": "cityhall", "vip": True, "prefered_languaje": "Italian", "role": "analist"},
    {"name": "Leo Orange", "phone": "555-234-6789", "email": "leo.orange@cityhall.com", "company": "cityhall", "vip": False, "prefered_languaje": "Italian", "role": "pm"},
    {"name": "Peter Bronze", "phone": "555-678-0123", "email": "peter.bronze@cityhall.com", "company": "cityhall", "vip": True, "prefered_languaje": "Italian", "role": "cto"},
    {"name": "Tom Tin", "phone": "555-012-4567", "email": "tom.tin@cityhall.com", "company": "cityhall", "vip": False, "prefered_languaje": "Italian", "role": "arquitect"},
    {"name": "Alice Smith", "phone": "555-123-4567", "email": "alice.smith@intermon.com", "company": "intermon", "vip": True, "prefered_languaje": "English", "role": "cto"},
    {"name": "Eve Davis", "phone": "555-567-8901", "email": "eve.davis@intermon.com", "company": "intermon", "vip": False, "prefered_languaje": "English", "role": "arquitect"},
    {"name": "Ivy Blue", "phone": "555-901-2345", "email": "ivy.blue@intermon.com", "company": "intermon", "vip": False, "prefered_languaje": "English", "role": "technician"},
    {"name": "Mia Purple", "phone": "555-345-7890", "email": "mia.purple@intermon.com", "company": "intermon", "vip": True, "prefered_languaje": "English", "role": "analist"},
    {"name": "Quinn Copper", "phone": "555-789-1234", "email": "quinn.copper@intermon.com", "company": "intermon", "vip": False, "prefered_languaje": "English", "role": "pm"},
    {"name": "Bob Johnson", "phone": "555-234-5678", "email": "bob.johnson@redcross.com", "company": "redcross", "vip": False, "prefered_languaje": "Spanish", "role": "pm"},
    {"name": "Frank White", "phone": "555-678-9012", "email": "frank.white@redcross.com", "company": "redcross", "vip": True, "prefered_languaje": "Spanish", "role": "cto"},
    {"name": "Jack Gray", "phone": "555-012-3456", "email": "jack.gray@redcross.com", "company": "redcross", "vip": False, "prefered_languaje": "Spanish", "role": "arquitect"},
    {"name": "Noah Silver", "phone": "555-456-8901", "email": "noah.silver@redcross.com", "company": "redcross", "vip": False, "prefered_languaje": "Spanish", "role": "technician"},
    {"name": "Ryan Iron", "phone": "555-890-2345", "email": "ryan.iron@redcross.com", "company": "redcross", "vip": True, "prefered_languaje": "Spanish", "role": "analist"},
    {"name": "Charlie Brown", "phone": "555-345-6789", "email": "charlie.brown@savethechildren.com", "company": "savethechildren", "vip": True, "prefered_languaje": "French", "role": "analist"},
    {"name": "Grace Black", "phone": "555-789-0123", "email": "grace.black@savethechildren.com", "company": "savethechildren", "vip": False, "prefered_languaje": "French", "role": "pm"},
    {"name": "Karen Lime", "phone": "555-123-5678", "email": "karen.lime@savethechildren.com", "company": "savethechildren", "vip": True, "prefered_languaje": "French", "role": "cto"},
    {"name": "Olivia Gold", "phone": "555-567-9012", "email": "olivia.gold@savethechildren.com", "company": "savethechildren", "vip": False, "prefered_languaje": "French", "role": "arquitect"},
    {"name": "Sophia Steel", "phone": "555-901-3456", "email": "sophia.steel@savethechildren.com", "company": "savethechildren", "vip": False, "prefered_languaje": "French", "role": "technician"}
]

roles = ["cto", "pm", "analist", "technician", "arquitect"]
random_stakeholders = list(map(lambda role: {"name": random.choice([p for p in stakeholders  if p["role"] == role])["name"], "role": role}, roles))

for project in projects:
    project["stakeholders"] = random_stakeholders

stakeholders_df = pd.DataFrame(stakeholders)
stakeholders_df.to_csv(os.path.join(workfolder, "stakeholders.csv"), index=False)
stakeholders_df = pd.read_csv(os.path.join(workfolder, "stakeholders.csv"))



In [ ]:
import json
import random

def get_random_stakeholder(project):
    stakeholders = project['stakeholders']
    stakeholders = stakeholders.replace("'", '"')
    stakeholders = json.loads(stakeholders)
    stakeholder = random.choice(stakeholders)
    return stakeholders_df[stakeholders_df["name"]==stakeholder["name"]].reset_index(drop=True).iloc[0].to_dict()

get_random_stakeholder(projects_df.iloc[0])